<a href="https://colab.research.google.com/github/eli576/Other/blob/main/Taller_CC_Deber_06_try_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install deap
import deap
from deap import base, creator, tools, algorithms
import random
import numpy as np
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 3.9 MB/s eta 0:00:00


In [8]:
# DATOS DEL PROBLEMA

#Aulas: {id: capacidad}
AULAS = {1: 15, 2: 30, 3: 20, 4: 25}

#Franjas Horarias (1-15)
FRANJAS = {
    1: "Lunes 9-11", 2: "Lunes 11-13", 3: "Lunes 13-15",
    4: "Martes 9-11", 5: "Martes 11-13", 6: "Martes 13-15",
    7: "Mierc 9-11", 8: "Mierc 11-13", 9: "Mierc 13-15",
    10: "Jueves 9-11", 11: "Jueves 11-13", 12: "Jueves 13-15",
    13: "Viernes 9-11", 14: "Viernes 11-13", 15: "Viernes 13-15"
}

# Profesores: {id: Nombre}
PROFESORES = {1: "Prof. García", 2: "Prof. Martínez", 3: "Prof. López", 4: "Prof. Ramos"}

# Asignaturas: {id: {nombre, [profesores_habilitados]}}
ASIGNATURAS = {
    1: {'nombre': 'CS1', 'profs': [1, 2]},
    2: {'nombre': 'IN1', 'profs': [1, 3]},
    3: {'nombre': 'MAI', 'profs': [1, 2]},
    4: {'nombre': 'FIS1', 'profs': [3, 4]},
    5: {'nombre': 'HIA', 'profs': [4]},
    6: {'nombre': 'DRA', 'profs': [1, 4]}
}

# Grupos: {id: {num_estudiantes, [asignaturas]}}
GRUPOS = {
    1: {'size': 10, 'subs': [1, 3, 4]},
    2: {'size': 30, 'subs': [2, 3, 5, 6]},
    3: {'size': 18, 'subs': [3, 4, 5]},
    4: {'size': 25, 'subs': [1, 4]},
    5: {'size': 20, 'subs': [2, 3, 5]},
    6: {'size': 22, 'subs': [1, 4, 5]},
    7: {'size': 16, 'subs': [1, 3]},
    8: {'size': 18, 'subs': [2, 6]},
    9: {'size': 24, 'subs': [1, 6]},
    10: {'size': 25, 'subs': [3, 4]}
}


# CLASES
# Generar la lista plana de "Eventos" (Total 26)
EVENTOS_A_PROGRAMAR = []
for g_id, g_data in GRUPOS.items():
    for sub_id in g_data['subs']:
        EVENTOS_A_PROGRAMAR.append({'grupo': g_id, 'materia': sub_id, 'size': g_data['size']})

NUM_EVENTOS = len(EVENTOS_A_PROGRAMAR)


# CONFIGURACION DE DEAP

# Maximizamos fitness (buscamos llegar a 0 desde negativos)
if hasattr(creator, "FitnessMax"):
    del creator.FitnessMax
if hasattr(creator, "Individual"):
    del creator.Individual

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Generación de Individuos
def gen_individual():
    ind = []
    for evento in range(NUM_EVENTOS):
        ind.append(random.randint(1, 4))  # Aula
        ind.append(random.randint(1, 15))  # Franja
        ind.append(random.randint(1, 4))  # Profesor
    return creator.Individual(ind)

toolbox.register("individual", gen_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# FUNCIÓN DE FITNESS

def evaluar_horario(individual):
    penalizacion = 0

    horario_profesor = {}
    horario_aula = {}
    horario_grupo = {}

    for i in range(NUM_EVENTOS):
        idx = i * 3
        aula_gen = individual[idx]
        franja_gen = individual[idx + 1]
        prof_gen = individual[idx + 2]

        info_evento = EVENTOS_A_PROGRAMAR[i]
        grupo_id = info_evento['grupo']
        materia_id = info_evento['materia']
        size_grupo = info_evento['size']

        # --- RESTRICCIONES DURAS (-100) ---

        # 1. Capacidad Aula
        if AULAS[aula_gen] < size_grupo:
            penalizacion -= 100

        # 2. Profesor Cualificado
        if prof_gen not in ASIGNATURAS[materia_id]['profs']:
            penalizacion -= 100

        # 3. Conflictos de Horario (Unicidad)

        # Profesor ocupado
        key_prof = (prof_gen, franja_gen)
        if key_prof in horario_profesor:
            penalizacion -= 100
        else:
            horario_profesor[key_prof] = 1

        # Aula ocupada
        key_aula = (aula_gen, franja_gen)
        if key_aula in horario_aula:
            penalizacion -= 100
        else:
            horario_aula[key_aula] = 1

        # Grupo ocupado
        key_grupo = (grupo_id, franja_gen)
        if key_grupo in horario_grupo:
            penalizacion -= 100
        else:
            horario_grupo[key_grupo] = 1

        # --- RESTRICCIONES BLANDAS (-1) ---
        # Horarios consecutivos para el profesor
        if (prof_gen, franja_gen + 1) in horario_profesor:
            penalizacion -= 1

    return (penalizacion,)

toolbox.register("evaluate", evaluar_horario)


# CONFIGURACION OPERADORES GENETICOS

toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxTwoPoint)


# Generamos listas completas de límites del tamaño exacto del individuo (78 genes)
low_bounds = [1, 1, 1] * NUM_EVENTOS  # [1, 1, 1, 1, 1, 1, ... x78]
up_bounds = [4, 15, 4] * NUM_EVENTOS  # [4, 15, 4, 4, 15, 4, ... x78]
toolbox.register("mutate", tools.mutUniformInt, low=low_bounds, up=up_bounds, indpb=0.05)



# EJECUCIÓN DEL ALGORITMO GENETICO

def main():
    random.seed(42)

    POBLACION = 400
    GENERACIONES = 100
    PROB_CRUCE = 0.7
    PROB_MUTACION = 0.2

    pop = toolbox.population(n=POBLACION)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=PROB_CRUCE, mutpb=PROB_MUTACION,
                                   ngen=GENERACIONES, stats=stats, halloffame=hof, verbose=True)

    mejor_ind = hof[0]
    print(f"\nMejor Fitness encontrado: {mejor_ind.fitness.values[0]}")

    return mejor_ind


# VISUALIZACION RESULTADOS

def imprimir_horario(individual):
    schedule_data = []

    for i in range(NUM_EVENTOS):
        idx = i * 3
        aula = individual[idx]
        franja = individual[idx + 1]
        prof = individual[idx + 2]

        evento = EVENTOS_A_PROGRAMAR[i]

        texto_franja = FRANJAS[franja]
        parts = texto_franja.split(" ", 1)
        dia = parts[0]
        hora = parts[1] if len(parts) > 1 else ""

        es_valido = prof in ASIGNATURAS[evento["materia"]]["profs"]
        mark = "" if es_valido else "(!)"

        schedule_data.append({
            "Asignatura": ASIGNATURAS[evento["materia"]]["nombre"],
            "Profesor": f"{PROFESORES[prof]} {mark}",
            "Grupo": evento["grupo"],
            "Aula": f"Aula {aula}",
            "Día": dia,
            "Hora": hora,
            "Franja": franja
        })

    # Convert to DataFrame
    df = pd.DataFrame(schedule_data)

    # Sort rows by franja
    df = df.sort_values(by="Franja").reset_index(drop=True)

    print("\nHORARIO OPTIMIZADO:\n")
    print(df)

    return df

mejor_solucion = main()
df_resultado = imprimir_horario(mejor_solucion)

gen	nevals	avg     	min  	max  
0  	400   	-3478.95	-4907	-2404
1  	294   	-3176.18	-4505	-2204
2  	316   	-2934.03	-4103	-2004
3  	327   	-2742.12	-3804	-1905
4  	300   	-2557.53	-3503	-1806
5  	311   	-2408.56	-3205	-1604
6  	307   	-2260.08	-3304	-1505
7  	298   	-2144.44	-3106	-1304
8  	310   	-2003.95	-2904	-1204
9  	304   	-1882.59	-2703	-1204
10 	307   	-1756.3 	-2710	-1202
11 	316   	-1653.88	-2603	-1002
12 	311   	-1571.78	-2402	-1002
13 	302   	-1471.87	-2307	-903 
14 	307   	-1347.26	-2303	-707 
15 	318   	-1242.05	-2003	-703 
16 	289   	-1153.93	-2404	-602 
17 	306   	-1090.87	-2102	-509 
18 	301   	-1002.03	-1903	-503 
19 	284   	-889.482	-1802	-501 
20 	308   	-816.835	-2006	-403 
21 	316   	-767.812	-1706	-403 
22 	294   	-706.21 	-1605	-403 
23 	280   	-651.15 	-1404	-302 
24 	308   	-586.765	-1303	-302 
25 	295   	-550.393	-1402	-205 
26 	322   	-528.45 	-1405	-204 
27 	292   	-483.82 	-1502	-203 
28 	311   	-461.627	-1603	-202 
29 	320   	-427.065	-1402	-103 
30 	297 